# REDES NEURAIS ARTIFICIAIS
## Evaluate Improve e Tunning

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [4]:
tf.__version__

'2.8.0'

### Importar o dataset

In [5]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head(50)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [6]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [7]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
np.shape(X_train)

(8000, 12)

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [11]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [12]:
ann.add(tf.keras.layers.Dense(units=6, activation='softplus'))

### Adding the second hidden layer

In [13]:
ann.add(tf.keras.layers.Dense(units=6, activation='softplus'))

### Adding the output layer

In [14]:
ann.add(tf.keras.layers.Dense(units=1, activation='tanh'))

## Part 3 - Training the ANN

### Compiling the ANN

In [15]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [16]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 8s 5ms/step - loss: 1.1054 - accuracy: 0.6152
Epoch 2/100
250/250 [==============================] - 1s 6ms/step - loss: 0.6418 - accuracy: 0.7311
Epoch 3/100
250/250 [==============================] - 1s 5ms/step - loss: 0.5845 - accuracy: 0.7725
Epoch 4/100
250/250 [==============================] - 2s 6ms/step - loss: 0.5644 - accuracy: 0.7461
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 0.5355 - accuracy: 0.7979
Epoch 6/100
250/250 [==============================] - 1s 6ms/step - loss: 0.5197 - accuracy: 0.7995
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 0.5242 - accuracy: 0.8012
Epoch 8/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5033 - accuracy: 0.8001
Epoch 9/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5508 - accuracy: 0.7451
Epoch 10/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4866 - accura

250/250 [==============================] - 1s 3ms/step - loss: 0.4005 - accuracy: 0.8231
Epoch 82/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3978 - accuracy: 0.8265
Epoch 83/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4002 - accuracy: 0.8288
Epoch 84/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3957 - accuracy: 0.8303
Epoch 85/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4333 - accuracy: 0.8119
Epoch 86/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4155 - accuracy: 0.8267
Epoch 87/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3990 - accuracy: 0.8301
Epoch 88/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4015 - accuracy: 0.8326
Epoch 89/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4919 - accuracy: 0.7809
Epoch 90/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4143 - accuracy: 

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [17]:
y_pred = ann.predict(X_test)
y_pred

array([[0.22037695],
       [0.36060607],
       [0.17516871],
       ...,
       [0.2549338 ],
       [0.1877955 ],
       [0.30085292]], dtype=float32)

In [18]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [19]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1562   33]
 [ 282  123]]


---
# Avaliando o Modelo

In [20]:
# Evaluating the ANN
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [21]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'softplus', input_dim = 12))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'softplus'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [22]:
classifier = KerasClassifier(build_fn = build_classifier,batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

C:\Users\MICKAE~1\AppData\Local\Temp/ipykernel_4984/4264426667.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn = build_classifier,batch_size = 10, epochs = 100)


In [23]:
print(mean,variance)

0.8287500083446503 0.031339865846046526


# Melhorando o modelo

In [24]:
# Improving the ANN
# Dropout Regularization to reduce overfitting if needed
from tensorflow.keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'softplus', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'softplus'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [25]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

C:\Users\MICKAE~1\AppData\Local\Temp/ipykernel_4984/1297209367.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)


In [26]:
print(mean,variance)

0.8084999978542328 0.007348471142600761


# Tuning the ANN

In [27]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'softplus', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'softplus'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {
    'batch_size': [10, 25, 32],
    'epochs': [50, 100],
    'optimizer': ['adam', 'rmsprop']
}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

C:\Users\MICKAE~1\AppData\Local\Temp/ipykernel_4984/669579587.py:16: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn = build_classifier)


In [28]:
%%time

grid_search = grid_search.fit(X_train, y_train)

Epoch 1/50
720/720 [==============================] - 3s 2ms/step - loss: 0.5084 - accuracy: 0.7972
Epoch 2/50
720/720 [==============================] - 1s 2ms/step - loss: 0.4572 - accuracy: 0.8015
Epoch 3/50
720/720 [==============================] - 1s 2ms/step - loss: 0.4445 - accuracy: 0.8043
Epoch 4/50
720/720 [==============================] - 1s 2ms/step - loss: 0.4455 - accuracy: 0.8032
Epoch 5/50
720/720 [==============================] - 1s 2ms/step - loss: 0.4416 - accuracy: 0.8069
Epoch 6/50
720/720 [==============================] - 1s 2ms/step - loss: 0.4388 - accuracy: 0.8058
Epoch 7/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4387 - accuracy: 0.8032
Epoch 8/50
720/720 [==============================] - 1s 2ms/step - loss: 0.4389 - accuracy: 0.8044
Epoch 9/50
720/720 [==============================] - 1s 2ms/step - loss: 0.4378 - accuracy: 0.8064
Epoch 10/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4338 - accuracy: 0.8075

720/720 [==============================] - 2s 2ms/step - loss: 0.3955 - accuracy: 0.8271
Epoch 33/50
720/720 [==============================] - 2s 3ms/step - loss: 0.4043 - accuracy: 0.8268
Epoch 34/50
720/720 [==============================] - 2s 3ms/step - loss: 0.3934 - accuracy: 0.8333
Epoch 35/50
720/720 [==============================] - 2s 3ms/step - loss: 0.3917 - accuracy: 0.8311
Epoch 36/50
720/720 [==============================] - 2s 3ms/step - loss: 0.4037 - accuracy: 0.8293
Epoch 37/50
720/720 [==============================] - 2s 3ms/step - loss: 0.3934 - accuracy: 0.8329
Epoch 38/50
720/720 [==============================] - 2s 2ms/step - loss: 0.3915 - accuracy: 0.8332
Epoch 39/50
720/720 [==============================] - 2s 2ms/step - loss: 0.3893 - accuracy: 0.8293
Epoch 40/50
720/720 [==============================] - 2s 2ms/step - loss: 0.3897 - accuracy: 0.8326
Epoch 41/50
720/720 [==============================] - 2s 2ms/step - loss: 0.3914 - accuracy: 0.8322
Ep

720/720 [==============================] - 2s 2ms/step - loss: 0.4271 - accuracy: 0.8082
Epoch 14/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4243 - accuracy: 0.8092
Epoch 15/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4184 - accuracy: 0.8107
Epoch 16/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4151 - accuracy: 0.8142
Epoch 17/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4114 - accuracy: 0.8169
Epoch 18/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4204 - accuracy: 0.8140
Epoch 19/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4041 - accuracy: 0.8178
Epoch 20/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4083 - accuracy: 0.8201
Epoch 21/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4038 - accuracy: 0.8185
Epoch 22/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4108 - accuracy: 0.8193
Ep

720/720 [==============================] - 2s 2ms/step - loss: 0.3814 - accuracy: 0.8382
Epoch 45/50
720/720 [==============================] - 2s 2ms/step - loss: 0.3787 - accuracy: 0.8374
Epoch 46/50
720/720 [==============================] - 2s 2ms/step - loss: 0.3763 - accuracy: 0.8429
Epoch 47/50
720/720 [==============================] - 2s 2ms/step - loss: 0.3800 - accuracy: 0.8415
Epoch 48/50
720/720 [==============================] - 2s 2ms/step - loss: 0.3814 - accuracy: 0.8396
Epoch 49/50
720/720 [==============================] - 2s 2ms/step - loss: 0.3797 - accuracy: 0.8421
Epoch 50/50
720/720 [==============================] - 2s 3ms/step - loss: 0.3784 - accuracy: 0.8388
Epoch 1/50
720/720 [==============================] - 2s 2ms/step - loss: 0.6254 - accuracy: 0.7944
Epoch 2/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4757 - accuracy: 0.7960
Epoch 3/50
720/720 [==============================] - 2s 2ms/step - loss: 0.4512 - accuracy: 0.8008
Epoch

720/720 [==============================] - 2s 2ms/step - loss: 3.1321 - accuracy: 0.7969
Epoch 26/50
720/720 [==============================] - 2s 3ms/step - loss: 3.1321 - accuracy: 0.7969
Epoch 27/50
720/720 [==============================] - 2s 2ms/step - loss: 3.1321 - accuracy: 0.7969
Epoch 28/50
720/720 [==============================] - 2s 2ms/step - loss: 3.1321 - accuracy: 0.7969
Epoch 29/50
720/720 [==============================] - 2s 2ms/step - loss: 3.1321 - accuracy: 0.7969
Epoch 30/50
720/720 [==============================] - 2s 3ms/step - loss: 3.1321 - accuracy: 0.7969
Epoch 31/50
720/720 [==============================] - 2s 3ms/step - loss: 3.1321 - accuracy: 0.7969
Epoch 32/50
720/720 [==============================] - 2s 3ms/step - loss: 3.1321 - accuracy: 0.7969
Epoch 33/50
720/720 [==============================] - 2s 3ms/step - loss: 3.1321 - accuracy: 0.7969
Epoch 34/50
720/720 [==============================] - 2s 3ms/step - loss: 3.1321 - accuracy: 0.7969
Ep

...

225/225 [==============================] - 0s 2ms/step - loss: 0.4389 - accuracy: 0.8079
Epoch 16/100
225/225 [==============================] - 0s 2ms/step - loss: 0.4438 - accuracy: 0.8051
Epoch 17/100
225/225 [==============================] - 1s 2ms/step - loss: 0.4417 - accuracy: 0.8037
Epoch 18/100
225/225 [==============================] - 0s 2ms/step - loss: 0.4332 - accuracy: 0.8071
Epoch 19/100
225/225 [==============================] - 1s 2ms/step - loss: 0.4360 - accuracy: 0.8051
Epoch 20/100
225/225 [==============================] - 1s 2ms/step - loss: 0.4337 - accuracy: 0.8085
Epoch 21/100
225/225 [==============================] - 1s 2ms/step - loss: 0.4373 - accuracy: 0.8065
Epoch 22/100
225/225 [==============================] - 1s 2ms/step - loss: 0.4309 - accuracy: 0.8040
Epoch 23/100
225/225 [==============================] - 1s 2ms/step - loss: 0.4274 - accuracy: 0.8067
Epoch 24/100
225/225 [==============================] - 0s 2ms/step - loss: 0.4329 - accuracy: 

225/225 [==============================] - 0s 2ms/step - loss: 0.3889 - accuracy: 0.8363
Epoch 96/100
225/225 [==============================] - 0s 2ms/step - loss: 0.3886 - accuracy: 0.8388
Epoch 97/100
225/225 [==============================] - 0s 2ms/step - loss: 0.4061 - accuracy: 0.8349
Epoch 98/100
225/225 [==============================] - 0s 2ms/step - loss: 0.3960 - accuracy: 0.8356
Epoch 99/100
225/225 [==============================] - 0s 2ms/step - loss: 0.3894 - accuracy: 0.8372
Epoch 100/100
225/225 [==============================] - 0s 2ms/step - loss: 0.4044 - accuracy: 0.8332
Epoch 1/100
225/225 [==============================] - 2s 2ms/step - loss: 2.7363 - accuracy: 0.7957
Epoch 2/100
225/225 [==============================] - 1s 2ms/step - loss: 0.5534 - accuracy: 0.7957
Epoch 3/100
225/225 [==============================] - 1s 2ms/step - loss: 0.4939 - accuracy: 0.7957
Epoch 4/100
225/225 [==============================] - 1s 2ms/step - loss: 0.4712 - accuracy: 0.7

225/225 [==============================] - 0s 2ms/step - loss: 0.3919 - accuracy: 0.8392
Epoch 76/100
225/225 [==============================] - 0s 2ms/step - loss: 0.3892 - accuracy: 0.8401
Epoch 77/100
225/225 [==============================] - 0s 2ms/step - loss: 0.3807 - accuracy: 0.8410
Epoch 78/100
225/225 [==============================] - 1s 3ms/step - loss: 0.3834 - accuracy: 0.8365
Epoch 79/100
225/225 [==============================] - 1s 3ms/step - loss: 0.3860 - accuracy: 0.8368
Epoch 80/100
225/225 [==============================] - 1s 3ms/step - loss: 0.3930 - accuracy: 0.8426
Epoch 81/100
225/225 [==============================] - 1s 2ms/step - loss: 0.3917 - accuracy: 0.8460
Epoch 82/100
225/225 [==============================] - 1s 2ms/step - loss: 0.3914 - accuracy: 0.8403
Epoch 83/100
225/225 [==============================] - 0s 2ms/step - loss: 0.3850 - accuracy: 0.8426
Epoch 84/100
225/225 [==============================] - 0s 2ms/step - loss: 0.3772 - accuracy: 

225/225 [==============================] - 0s 2ms/step - loss: 0.3936 - accuracy: 0.8328
Epoch 56/100
225/225 [==============================] - 0s 2ms/step - loss: 0.4065 - accuracy: 0.8372
Epoch 57/100
225/225 [==============================] - 0s 2ms/step - loss: 0.3996 - accuracy: 0.8390
Epoch 58/100
225/225 [==============================] - 0s 2ms/step - loss: 0.3970 - accuracy: 0.8415
Epoch 59/100
225/225 [==============================] - 0s 2ms/step - loss: 0.4016 - accuracy: 0.8350
Epoch 60/100
225/225 [==============================] - 0s 2ms/step - loss: 0.4049 - accuracy: 0.8389
Epoch 61/100
225/225 [==============================] - 0s 2ms/step - loss: 0.3936 - accuracy: 0.8361
Epoch 62/100
225/225 [==============================] - 0s 2ms/step - loss: 0.4012 - accuracy: 0.8401
Epoch 63/100
225/225 [==============================] - 0s 2ms/step - loss: 0.3960 - accuracy: 0.8379
Epoch 64/100
225/225 [==============================] - 0s 2ms/step - loss: 0.3932 - accuracy: 

800/800 [==============================] - 2s 2ms/step - loss: 3.1467 - accuracy: 0.7960
Epoch 36/100
800/800 [==============================] - 2s 2ms/step - loss: 3.1467 - accuracy: 0.7960
Epoch 37/100
800/800 [==============================] - 2s 2ms/step - loss: 3.1467 - accuracy: 0.7960
Epoch 38/100
800/800 [==============================] - 2s 2ms/step - loss: 3.1467 - accuracy: 0.7960
Epoch 39/100
800/800 [==============================] - 2s 2ms/step - loss: 3.1467 - accuracy: 0.7960
Epoch 40/100
800/800 [==============================] - 2s 2ms/step - loss: 3.1467 - accuracy: 0.7960
Epoch 41/100
800/800 [==============================] - 2s 2ms/step - loss: 3.1467 - accuracy: 0.7960
Epoch 42/100
800/800 [==============================] - 2s 2ms/step - loss: 3.1467 - accuracy: 0.7960
Epoch 43/100
800/800 [==============================] - 2s 2ms/step - loss: 3.1467 - accuracy: 0.7960
Epoch 44/100
800/800 [==============================] - 2s 2ms/step - loss: 3.1467 - accuracy: 

In [29]:
print('Best params:\n',grid_search.best_params_,'\nBest accuracy:\n',grid_search.best_score_)

Best params:
 {'batch_size': 10, 'epochs': 100, 'optimizer': 'adam'} 
Best accuracy:
 0.8455


# Halving Grid Search

In [30]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'softplus', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'softplus'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {
    'batch_size': [10, 25, 32],
    'epochs': [50, 100],
    'optimizer': ['adam', 'rmsprop']
}

halving_cv = HalvingGridSearchCV(estimator = classifier,
                                  param_grid = parameters,
                                  scoring="accuracy", 
                                  n_jobs=-1, 
                                  min_resources="exhaust", 
                                  factor=3,
                                  cv = 10)

C:\Users\MICKAE~1\AppData\Local\Temp/ipykernel_4984/1768710774.py:17: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn = build_classifier)


In [31]:
%%time
halving_cv = halving_cv.fit(X_train, y_train)

Epoch 1/100
320/320 [==============================] - 2s 2ms/step - loss: 0.5171 - accuracy: 0.7960
Epoch 2/100
320/320 [==============================] - 1s 2ms/step - loss: 0.4768 - accuracy: 0.7965
Epoch 3/100
320/320 [==============================] - 1s 2ms/step - loss: 0.4525 - accuracy: 0.8043
Epoch 4/100
320/320 [==============================] - 1s 2ms/step - loss: 0.4477 - accuracy: 0.8071
Epoch 5/100
320/320 [==============================] - 1s 2ms/step - loss: 0.4420 - accuracy: 0.8073
Epoch 6/100
320/320 [==============================] - 1s 2ms/step - loss: 0.4377 - accuracy: 0.8073
Epoch 7/100
320/320 [==============================] - 1s 2ms/step - loss: 0.4369 - accuracy: 0.8056
Epoch 8/100
320/320 [==============================] - 1s 2ms/step - loss: 0.4375 - accuracy: 0.8075
Epoch 9/100
320/320 [==============================] - 1s 2ms/step - loss: 0.4406 - accuracy: 0.8027
Epoch 10/100
320/320 [==============================] - 1s 2ms/step - loss: 0.4347 - accura

320/320 [==============================] - 1s 3ms/step - loss: 0.3845 - accuracy: 0.8371
Epoch 82/100
320/320 [==============================] - 1s 2ms/step - loss: 0.3871 - accuracy: 0.8376
Epoch 83/100
320/320 [==============================] - 1s 2ms/step - loss: 0.3774 - accuracy: 0.8411
Epoch 84/100
320/320 [==============================] - 1s 2ms/step - loss: 0.3787 - accuracy: 0.8382
Epoch 85/100
320/320 [==============================] - 1s 2ms/step - loss: 0.3786 - accuracy: 0.8397
Epoch 86/100
320/320 [==============================] - 1s 2ms/step - loss: 0.3812 - accuracy: 0.8403
Epoch 87/100
320/320 [==============================] - 1s 2ms/step - loss: 0.3778 - accuracy: 0.8375
Epoch 88/100
320/320 [==============================] - 1s 2ms/step - loss: 0.3765 - accuracy: 0.8382
Epoch 89/100
320/320 [==============================] - 1s 2ms/step - loss: 0.3727 - accuracy: 0.8419
Epoch 90/100
320/320 [==============================] - 1s 2ms/step - loss: 0.3751 - accuracy: 

In [32]:
print('Best params:\n',halving_cv.best_params_,'\nBest accuracy:\n',halving_cv.best_score_)

Best params:
 {'batch_size': 25, 'epochs': 100, 'optimizer': 'adam'} 
Best accuracy:
 0.8448060075093867


# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)